In [1]:
!wget https://github.com/chugharun/Translationsubtext/raw/main/data_en_hi.zip

--2021-05-17 05:32:25--  https://github.com/chugharun/Translationsubtext/raw/main/data_en_hi.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chugharun/Translationsubtext/main/data_en_hi.zip [following]
--2021-05-17 05:32:25--  https://raw.githubusercontent.com/chugharun/Translationsubtext/main/data_en_hi.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6083712 (5.8M) [application/zip]
Saving to: ‘data_en_hi.zip’

data_en_hi.zip      100%[===================>]   5.80M  --.-KB/s    in 0.07s   

2021-05-17 05:32:25 (88.7 MB/s) - ‘data_en_hi.zip’ saved [6083712/6083712]



In [2]:
!unzip /content/data_en_hi.zip

Archive:  /content/data_en_hi.zip
  inflating: train_en                
  inflating: val_hi                  
  inflating: val_en                  
  inflating: test_hi                 
  inflating: test_en                 
  inflating: train_hi                


In [3]:
!pip install -U spacy

     |████████████████████████████████| 12.8MB 24.9MB/s 
     |████████████████████████████████| 1.1MB 52.8MB/s 
     |████████████████████████████████| 9.1MB 149kB/s 
     |████████████████████████████████| 460kB 47.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 92kB 14.1MB/s 
     |████████████████████████████████| 122kB 60.0MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=cd132d8b5ead69b0635377794b884eeba249877f47056735922dc9427aeb59a7
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.

In [4]:
!python -m spacy download en_core_web_sm

2021-05-17 05:32:53.304441: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.7MB 224kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
!pip install indic-nlp-library
from indicnlp.tokenize import indic_tokenize

In [6]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
from torch import Tensor
import io
import time

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)


#url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
#train_urls = ('train.de.gz', 'train.en.gz')
#val_urls = ('val.de.gz', 'val.en.gz')
#test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

#train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
#val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
#test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

#de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
train_filepaths = ['/content/train_hi','/content/train_en']
val_filepaths = ['/content/val_hi','/content/val_en']
test_filepaths =['/content/test_hi','/content/test_en']
#hi_vocab = build_vocab(train_filepaths[0], de_tokenizer)
hi_tokenizer = indic_tokenize.trivial_tokenize
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_.lower()))
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

hi_vocab = build_vocab(train_filepaths[0], hi_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

def data_process(filepaths):
  raw_hi_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_hi, raw_en) in zip(raw_hi_iter, raw_en_iter):
    hi_tensor_ = torch.tensor([hi_vocab[token] for token in hi_tokenizer(raw_hi.rstrip("\n"))],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en.rstrip("\n"))],
                            dtype=torch.long)
    data.append((hi_tensor_, en_tensor_))
  return data


train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


BATCH_SIZE = 16
PAD_IDX = en_vocab['<pad>']
BOS_IDX = en_vocab['<bos>']
EOS_IDX = en_vocab['<eos>']

In [45]:
from torchtext.data.metrics import bleu_score 

def bleu(model, src,tgt, src_vocab, tgt_vocab, src_tokenizer):
  targets = []
  outputs = []
  for example in zip(src,tgt):
    source = example[0]
    trgt = example[1]
    prediction = translate(model, source, src_vocab, tgt_vocab, src_tokenizer)
    prediction = prediction[:-1]
    targets.append([trgt.split()])
    outputs.append(prediction.split())
    
  return bleu_score(outputs, targets)


In [10]:
len(en_vocab)

47364

In [11]:
len(hi_vocab)

65640

In [12]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  hi_batch, en_batch = [], []
  for (hi_item, en_item) in data_batch:
    hi_batch.append(torch.cat([torch.tensor([BOS_IDX]), hi_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  hi_batch = pad_sequence(hi_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return en_batch,hi_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [13]:
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)


class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.2):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [15]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [20]:
SRC_VOCAB_SIZE = len(en_vocab)
TGT_VOCAB_SIZE = len(hi_vocab)
EMB_SIZE = 400
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 16
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 20
#torch.cuda.empty_cache()
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.AdamW(
    transformer.parameters(), lr=0.0001,eps=1e-16, betas=(0.9,0.999)
)

In [21]:
transformer.eval()

Seq2SeqTransformer(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=400, out_features=400, bias=True)
        )
        (linear1): Linear(in_features=400, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=512, out_features=400, bias=True)
        (norm1): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=400, out_features=400, bias=True)
        )
        (linear1): Linear(in_features=400, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
   

In [22]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)


def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

In [23]:
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  val_loss = evaluate(transformer, valid_iter)
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f},Val loss: {val_loss:.3f} "
          f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 6.243,Val loss: 5.374 Epoch time = 392.369s
Epoch: 2, Train loss: 5.251,Val loss: 4.937 Epoch time = 393.487s
Epoch: 3, Train loss: 4.759,Val loss: 4.685 Epoch time = 393.894s
Epoch: 4, Train loss: 4.373,Val loss: 4.485 Epoch time = 396.424s
Epoch: 5, Train loss: 4.035,Val loss: 4.391 Epoch time = 396.508s
Epoch: 6, Train loss: 3.727,Val loss: 4.334 Epoch time = 394.947s
Epoch: 7, Train loss: 3.443,Val loss: 4.267 Epoch time = 394.949s
Epoch: 8, Train loss: 3.177,Val loss: 4.274 Epoch time = 394.428s
Epoch: 9, Train loss: 2.925,Val loss: 4.323 Epoch time = 394.730s
Epoch: 10, Train loss: 2.684,Val loss: 4.359 Epoch time = 393.577s
Epoch: 11, Train loss: 2.461,Val loss: 4.410 Epoch time = 393.811s
Epoch: 12, Train loss: 2.253,Val loss: 4.469 Epoch time = 393.524s
Epoch: 13, Train loss: 2.064,Val loss: 4.552 Epoch time = 393.869s
Epoch: 14, Train loss: 1.896,Val loss: 4.620 Epoch time = 393.734s
Epoch: 15, Train loss: 1.747,Val loss: 4.721 Epoch time = 394.944s
Epoc

In [24]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys


def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
  model.eval()
  src =src.lower()
  tokens = [BOS_IDX] + [src_vocab.stoi[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  return " ".join([tgt_vocab.itos[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [56]:
translate(transformer, "One of my boys needed to lose some weight.", en_vocab, hi_vocab, en_tokenizer)
#This sentence was part of review from bonus task

' एक के बच्चे को वजन कम करने की जरूरत है । '

In [51]:
with io.open('/content/train_en', encoding="utf8") as f:
  i=0
  for string_ in f:
    #print(string_)
    i+=1
print(i)

57158


In [47]:
with io.open('/content/test_en', encoding="utf8") as f:
  src_ =[]
  for i,string_ in enumerate(f):
    if i<100:
      src_.append(string_)
    else:
      break

In [48]:
with io.open('/content/test_hi', encoding="utf8") as f:
  tgt_ =[]
  for i,string_ in enumerate(f):
    if i<100:
      tgt_.append(string_)
    else:
      break

In [49]:
score = bleu(transformer,src_,tgt_,en_vocab, hi_vocab, en_tokenizer)
print(f"Test Bleu score {score * 100:.2f}")

Test Bleu score 18.35
